In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd

In [2]:
print(os.getcwd())
%cd ../../..

/Users/elizagoler/Documents/RecessionFertility/source/derived/county_gfrs
/Users/elizagoler/Documents/RecessionFertility


In [3]:
natality_df_68 = pd.read_csv('datastore/raw/nvss/data/natality1968us.csv')

In [8]:
natality_df_68.head()

,datayear,rectype,restatus,stateres,cntyres,cityres,citrspop,smsares,metrores,frace,...,dgestat,gestat10,dbirwt,birwt12,birwt3,dlegit,dplural,plural2,statenat,cntynat
0,8,1,1,1,1020,999,9,0,2,1,...,40,6,3232,7,2,1,1,1,1,1020
1,8,1,1,1,1001,999,9,0,2,1,...,40,6,2892,6,2,1,1,1,1,1001
2,8,1,1,1,1001,999,9,0,2,2,...,34,3,1985,4,1,1,1,1,1,1001
3,8,2,2,1,1026,999,9,0,2,1,...,40,6,3289,7,2,1,1,1,1,1001
4,8,1,1,1,1001,999,9,0,2,9,...,40,6,3742,8,2,1,1,1,1,1001


In [4]:
natality_df_68.columns

Index(['datayear', 'rectype', 'restatus', 'stateres', 'cntyres', 'cityres',
       'citrspop', 'smsares', 'metrores', 'frace', 'mrace', 'crace', 'crace2',
       'crace3', 'csex', 'birmon', 'dmage', 'mage36', 'mage8', 'mage12',
       'dlivord', 'livord9', 'livord3', 'birattnd', 'dgestat', 'gestat10',
       'dbirwt', 'birwt12', 'birwt3', 'dlegit', 'dplural', 'plural2',
       'statenat', 'cntynat'],
      dtype='object')

In [5]:
natality_df_68_subset = natality_df_68[['datayear', 'stateres', 'cntyres', 'dplural']].copy()
natality_df_68_subset

,datayear,stateres,cntyres,dplural
0,8,1,1020,1
1,8,1,1001,1
2,8,1,1001,1
3,8,1,1026,1
4,8,1,1001,1
...,...,...,...,...
1750777,8,51,51023,1
1750778,8,51,51023,1
1750779,8,51,51023,1
1750780,8,51,51009,1


In [13]:
# Print all unique cntyres codes
unique_cntyres = natality_df_68['cntyres'].unique()

In [14]:
natality_df_69 = pd.read_csv('datastore/raw/nber_nvss_natality/data/natality1969us.csv')

/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_23574/374703579.py:1: DtypeWarning: Columns (9,51) have mixed types. Specify dtype option on import or set low_memory=False.
  natality_df_69 = pd.read_csv('datastore/raw/nber_nvss_natality/data/natality1969us.csv')


In [15]:
natality_df_69.head()

,datayear,shipnum,reparea,rectype,restatus,stateres,cntyres,cityres,citrspop,smsares,...,legitsmsarflg,educsmsarflg,congenrflg,legitstoflg,educstoflg,lmpoflg,monpreoflg,llboflg,lfdoflg,congenflg
0,9,1,0,1,1,1,1001,999,9,000,...,0,0,1,1,0,0,0,0,0,1
1,9,1,0,2,2,1,1026,999,9,000,...,0,0,1,1,0,0,0,0,0,1
2,9,1,0,1,1,1,1001,999,9,000,...,0,0,1,1,0,0,0,0,0,1
3,9,1,0,1,1,1,1001,999,9,000,...,0,0,1,1,0,0,0,0,0,1
4,9,1,0,1,1,1,1001,999,9,000,...,0,0,1,1,0,0,0,0,0,1


In [16]:
natality_df_80 = pd.read_csv('datastore/raw/nber_nvss_natality/data/natality1980us.csv')

/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_23574/2837661266.py:1: DtypeWarning: Columns (1,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  natality_df_80 = pd.read_csv('datastore/raw/nber_nvss_natality/data/natality1980us.csv')


In [17]:
natality_df_80.head()

,datayear,shipnum,reparea,rectype,restatus,stateres,cntyres,cityres,citrspop,smsares,...,otermm20,omaps,omapsr,fmaps,fmapsr,origm,origf,recwt,nprev28,nprev12
0,0,6,0,1,1,1,01002,999,9,177,...,0,9,4,10,4,88,88,1,16,9
1,0,6,0,1,1,1,01002,999,9,177,...,0,9,4,10,4,88,88,1,21,11
2,0,6,0,1,1,1,01002,999,9,177,...,0,9,4,10,4,88,88,1,21,11
3,0,6,0,1,1,1,01002,999,9,177,...,0,2,1,1,1,88,88,1,1,1
4,0,6,0,1,1,1,01002,999,9,177,...,0,8,3,10,4,88,88,1,13,7


In [ ]:
# Read in each csv (can I just do specific columns to avoid loading the whole dataframe?)
# Repalce the year column with the full year of the data 
# Concatenate all the dataframes together 

In [ ]:
natality_all_years_df = pd.concat(natality_dfs, ignore_index=True)

In [ ]:
# Missing files are 1970, 1972, 1994, 1996, 2008

In [6]:
natality_df_2002 = pd.read_csv(
    'datastore/raw/nvss/data/natality2002us.csv',
    usecols=['datayear', 'stateres', 'cntyres', 'dplural']
).copy()


/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_71263/3914313814.py:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  natality_df_2002 = pd.read_csv(


In [10]:
len(natality_df_2002['cntyres'].unique())

1007

In [16]:
len(natality_df_2002)

4027376

In [27]:
births_1971.head()

,datayear,shipnum,reparea,rectype,restatus,stateres,cntyres,cityres,citrspop,smsares,...,legitsmsarflg,educsmsarflg,congenrflg,legitstoflg,educstoflg,lmpoflg,monpreoflg,llboflg,lfdoflg,congenflg
0,9,1,0,1,1,1,1001,999,9,000,...,0,0,1,1,0,0,0,0,0,1
1,9,1,0,2,2,1,1026,999,9,000,...,0,0,1,1,0,0,0,0,0,1
2,9,1,0,1,1,1,1001,999,9,000,...,0,0,1,1,0,0,0,0,0,1
3,9,1,0,1,1,1,1001,999,9,000,...,0,0,1,1,0,0,0,0,0,1
4,9,1,0,1,1,1,1001,999,9,000,...,0,0,1,1,0,0,0,0,0,1


In [30]:
filename_1980 = "datastore/raw/nvss/data/natality1980us.csv"
 
births_1980 = pd.read_csv(
    filename_1980,
    #usecols=['datayear', 'stateres', 'cntyres', 'dplural']
)


/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_71263/154311459.py:3: DtypeWarning: Columns (1,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  births_1980 = pd.read_csv(


In [31]:
births_1980.head()

,datayear,shipnum,reparea,rectype,restatus,stateres,cntyres,cityres,citrspop,smsares,...,otermm20,omaps,omapsr,fmaps,fmapsr,origm,origf,recwt,nprev28,nprev12
0,0,6,0,1,1,1,01002,999,9,177,...,0,9,4,10,4,88,88,1,16,9
1,0,6,0,1,1,1,01002,999,9,177,...,0,9,4,10,4,88,88,1,21,11
2,0,6,0,1,1,1,01002,999,9,177,...,0,9,4,10,4,88,88,1,21,11
3,0,6,0,1,1,1,01002,999,9,177,...,0,2,1,1,1,88,88,1,1,1
4,0,6,0,1,1,1,01002,999,9,177,...,0,8,3,10,4,88,88,1,13,7


In [4]:
data_dir = "datastore/raw/nvss/data"
natality_dfs_68_78 = []

for year in range(1968, 1979):  # inclusive 1968 through 1978
    filename = f"natality{year}us.csv"
    filepath = os.path.join(data_dir, filename)
    if os.path.exists(filepath):
        df = pd.read_csv(filepath)
        df['datayear'] = year  # ensure datayear matches the file year
        natality_dfs_68_78.append(df)
    else:
        print(f"File {filename} does not exist.")

# Now natality_dfs_68_78 holds the 1968-1978 dfs with correct datayear column.


/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_75471/1418554910.py:8: DtypeWarning: Columns (9,51) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_75471/1418554910.py:8: DtypeWarning: Columns (6,7,8,9,10,51) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_75471/1418554910.py:8: DtypeWarning: Columns (6,7,8,9,10,54) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_75471/1418554910.py:8: DtypeWarning: Columns (1,2,6,7,8,9,10,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_75471/1418554910.py:8: DtypeWarning: Columns (1,2,6,7,

In [5]:
print(len(natality_dfs_68_78))

11


In [18]:
natality_1968_df = natality_dfs_68_78[0]
natality_1968_df.columns

# Predictions:
# datayear: year 
# cntyres: mother's county of residence (as opposed to ctynat)
# cityres: mother's city of residence 
# Number: dplural

Index(['datayear', 'rectype', 'restatus', 'stateres', 'cntyres', 'cityres',
       'citrspop', 'smsares', 'metrores', 'frace', 'mrace', 'crace', 'crace2',
       'crace3', 'csex', 'birmon', 'dmage', 'mage36', 'mage8', 'mage12',
       'dlivord', 'livord9', 'livord3', 'birattnd', 'dgestat', 'gestat10',
       'dbirwt', 'birwt12', 'birwt3', 'dlegit', 'dplural', 'plural2',
       'statenat', 'cntynat'],
      dtype='object')

In [19]:
natality_1969_df = natality_dfs_68_78[1]
natality_1969_df.columns

# Predictions:
# datayear: year
# cntyres: mother's county of residence (as opposed to ctynat)
# cityres: mother's city of residence 
# Number: not in this df?

Index(['datayear', 'shipnum', 'reparea', 'rectype', 'restatus', 'stateres',
       'cntyres', 'cityres', 'citrspop', 'smsares', 'metrores', 'divres',
       'stsubres', 'statenat', 'cntynat', 'divocc', 'stsubocc', 'csex',
       'birattnd', 'frace', 'mrace', 'crace', 'crace3', 'dmage', 'mage36',
       'mage15', 'mage12', 'mage8', 'mage7', 'mage6', 'nlbnl', 'nlbnd', 'nlbd',
       'dtotord', 'totord9', 'dlivord', 'livord9', 'livord8', 'livord7',
       'livord6', 'livord3', 'flag77', 'dfage', 'fage11', 'dbirwt', 'birwt12',
       'birwt3', 'birmon', 'birday', 'lmpmon', 'lmpday', 'lmpyr', 'dgestat',
       'gestat10', 'gestat3', 'dmeduc', 'meduc14', 'meduc6', 'dfeduc',
       'feduc14', 'dlegit', 'legit3', 'monprec', 'mpre10', 'mpre6', 'llbmon',
       'llbyr', 'disllb', 'isllb17', 'isllb10', 'isllb8', 'lfdmon', 'lfdyr',
       'fdflg', 'dlfd', 'dterm', 'term9', 'outcome', 'congen', 'legitstrflg',
       'educstrflg', 'lmprflg', 'monprerflg', 'llbrflg', 'lfdrflg',
       'legitsmsarflg'

In [21]:
natality_1970_df = natality_dfs_68_78[2]
natality_1970_df.head()

# Predictions:
# datayear: year
# cntyres: mother's county of residence (as opposed to ctynat)
# cityres: mother's city of residence 
# Number: skipping in this round

,datayear,shipnum,reparea,rectype,restatus,stateres,cntyres,cityres,citrspop,smsares,metrores,divres,stsubres,statenat,cntynat,divocc,stsubocc,csex,birattnd,frace,mrace,crace,crace3,dmage,mage36,mage15,mage12,mage8,mage7,mage6,nlbnl,nlbnd,nlbd,dtotord,totord9,dlivord,livord9,livord8,livord7,livord6,livord3,flag77,dfage,fage11,dbirwt,birwt12,birwt3,birmon,birday,lmpmon,lmpday,lmpyr,dgestat,gestat10,gestat3,dmeduc,meduc14,meduc6,dfeduc,feduc14,dlegit,legit3,monprec,mpre10,mpre6,llbmon,llbyr,disllb,isllb17,isllb10,isllb8,lfdmon,lfdyr,fdflg,dlfd,dterm,term9,outcome,mplbir,congen,legitstrflg,educstrflg,lmprflg,monprerflg,llbrflg,lfdrflg,legitsmsarflg,educsmsarflg,congenrflg,legitstoflg,educstoflg,lmpoflg,monpreoflg,llboflg,lfdoflg,congenflg
0,1970,1,0,1,1,1,1001,999,9,0,2,6,63,1,1001,6,63,1,1,2,2,2,3,31,18,15,10,5,5,4,4,1,0,6,6,6,6,6,6,5,2,NaN,33,5,4876,10,2,1,16,88,88,8,0,0,0,88,0,0,88,0,1,1,&,0,NaN,88,88,888,0,0,0,88,88,0,888,888,0,0,1,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1
1,1970,1,0,1,1,1,1001,999,9,0,2,6,63,1,1001,6,63,2,1,1,1,1,1,18,5,6,6,2,2,1,0,0,0,1,1,1,1,1,1,1,1,NaN,20,3,3544,8,2,1,7,88,88,8,0,0,0,88,0,0,88,0,1,1,&,0,NaN,88,88,888,0,0,0,88,88,0,888,888,0,0,44,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1
2,1970,1,0,1,1,1,1001,999,9,0,2,6,63,1,1001,6,63,1,1,1,1,1,1,23,10,12,8,3,3,2,0,0,0,1,1,1,1,1,1,1,1,NaN,27,4,4111,9,2,1,23,88,88,8,0,0,0,88,0,0,88,0,1,1,&,0,NaN,88,88,888,0,0,0,88,88,0,888,888,0,0,1,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1
3,1970,1,0,2,2,1,1026,999,9,131,1,6,63,1,1001,6,63,1,1,2,2,2,3,41,28,17,12,7,7,6,9,0,0,10,8,10,8,7,6,5,2,NaN,51,9,3430,7,2,1,9,88,88,8,0,0,0,88,0,0,88,0,1,1,&,0,NaN,88,88,888,0,0,0,88,88,0,888,888,0,0,1,0,1,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1
4,1970,1,0,1,1,1,1001,999,9,0,2,6,63,1,1001,6,63,1,3,2,2,2,3,27,14,14,9,4,4,3,2,3,77,6,6,6,6,6,6,5,2,NaN,29,4,2948,6,2,1,24,88,88,8,0,0,0,88,0,0,88,0,1,1,&,0,NaN,88,88,888,0,0,0,88,88,0,888,888,0,0,1,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1


In [22]:
natality_1971_df = natality_dfs_68_78[3]
natality_1971_df.head()

,datayear,shipnum,reparea,rectype,restatus,stateres,cntyres,cityres,citrspop,smsares,metrores,divres,stsubres,statenat,cntynat,divocc,stsubocc,csex,birattnd,frace,mrace,crace,crace3,dmage,mage36,mage15,mage12,mage8,mage7,mage6,nlbnl,nlbnd,nlbd,dtotord,totord9,dlivord,livord9,livord8,livord7,livord6,livord3,flag77,dfage,fage11,dbirwt,birwt12,birwt3,dplural,plural3,plural2,birmon,birday,lmpmon,lmpday,lmpyr,dgestat,gestat10,gestat3,dmeduc,meduc14,meduc6,dfeduc,feduc14,dlegit,legit3,monprec,mpre10,mpre6,llbmon,llbyr,disllb,isllb17,isllb10,isllb8,lfdmon,lfdyr,dlfd,dterm,term9,outcome,mplbir,congen,legitstrflg,educstrflg,lmprflg,monprerflg,llbrflg,lfdrflg,legitsmsarflg,educsmsarflg,congenrflg,legitstoflg,educstoflg,lmpoflg,monpreoflg,llboflg,lfdoflg,congenflg
0,1971,1.0,0.0,1,1,1,1001,999,9,0,2,6,63,1,1001,6,63,1,1,1,1,1,1,20,7,9,8,3,3,2,0,0,0,1,1,1,1,1,1,1,1,NaN,22,3,4196,9,2,1,1,1,1,19,88,88,8,0,0,0,88,0,0,88,0,1,1,&,0,0,88,88,888,0,0,0,88,88,888,888,0,0,1,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1
1,1971,1.0,0.0,2,2,1,1002,999,9,128,1,6,63,1,1001,6,63,1,1,1,1,1,1,27,14,14,9,4,4,3,2,0,0,3,3,3,3,3,3,3,2,NaN,27,4,2920,6,2,1,1,1,1,19,88,88,8,0,0,0,88,0,0,88,0,1,1,&,0,0,88,88,888,0,0,0,88,88,888,888,0,0,1,0,1,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1
2,1971,1.0,0.0,1,1,1,1002,999,9,128,1,6,63,1,1002,6,63,2,1,2,2,2,3,20,7,9,8,3,3,2,2,0,0,3,3,3,3,3,3,3,2,NaN,30,5,2892,6,2,1,1,1,1,30,88,88,8,0,0,0,88,0,0,88,0,1,1,&,0,0,88,88,888,0,0,0,88,88,888,888,0,0,1,0,1,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1
3,1971,1.0,0.0,1,1,1,1002,999,9,128,1,6,63,1,1002,6,63,1,1,1,1,1,1,28,15,14,9,4,4,3,2,2,0,5,5,5,5,5,5,5,2,NaN,32,5,3232,7,2,1,1,1,1,6,88,88,8,0,0,0,88,0,0,88,0,1,1,&,0,0,88,88,888,0,0,0,88,88,888,888,0,0,1,0,1,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1
4,1971,1.0,0.0,1,1,1,1002,999,9,128,1,6,63,1,1002,6,63,1,1,1,1,1,1,30,17,15,10,5,5,4,2,0,0,3,3,3,3,3,3,3,2,NaN,26,4,3175,7,2,1,1,1,1,24,88,88,8,0,0,0,88,0,0,88,0,1,1,&,0,0,88,88,888,0,0,0,88,88,888,888,0,0,10,0,1,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1


In [23]:
natality_1972_df = natality_dfs_68_78[4]
natality_1972_df.head()

# Predictions:
# datayear: year 
# cntyres: mother's county of residence (as opposed to ctynat)
# cityres: mother's city of residence 
# Number: dplural

,datayear,shipnum,reparea,rectype,restatus,stateres,cntyres,cityres,citrspop,smsares,metrores,divres,stsubres,statenat,cntynat,divocc,stsubocc,csex,birattnd,frace,mrace,crace,crace3,dmage,mage36,mage15,mage12,mage8,mage7,mage6,nlbnl,nlbnd,nlbd,dtotord,totord9,dlivord,livord9,livord8,livord7,livord6,livord3,dfage,fage11,dbirwt,birwt12,birwt3,dplural,plural3,plural2,birmon,birday,lmpmon,lmpday,lmpyr,dgestat,gestat10,gestat3,dmeduc,meduc14,meduc6,dfeduc,feduc14,dlegit,legit3,monprec,mpre10,mpre6,llbmon,llbyr,disllb,isllb17,isllb10,isllb8,lfdmon,lfdyr,dlfd,dterm,term9,outcome,mplbir,nprevis,legitstrflg,educstrflg,lmprflg,monprerflg,llbrflg,lfdrflg,legitsmsarflg,educsmsarflg,congenrflg,nprevisrflg,legitstoflg,educstoflg,lmpoflg,monpreoflg,llboflg,lfdoflg,congenflg,nprevisoflg,recwt,nprev28
0,1972,1,0,1,1,1,01002,999,9,128,1,6,63,1,1002,6,63,2,1,1,1,1,1,32,19,15,10,5,5,4,7,0,0,8,8,8,8,7,6,5,2,26,4,4479,9,2,1,1,1,1,20,88,88,8,0,0,0,88,0,0,88,0,1,1,&,0,0,88,88,888,0,0,0,88,88,888,888,0,0,99,88,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,2,0
1,1972,1,0,1,1,1,01002,999,9,128,1,6,63,1,1002,6,63,1,1,1,1,1,1,16,3,4,4,2,2,1,0,0,0,1,1,1,1,1,1,1,1,19,2,3969,8,2,1,1,1,1,4,88,88,8,0,0,0,88,0,0,88,0,1,1,&,0,0,88,88,888,0,0,0,88,88,888,888,0,0,99,88,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,2,0
2,1972,1,0,1,1,1,01002,999,9,128,1,6,63,1,1002,6,63,1,1,1,1,1,1,34,21,15,10,5,5,4,1,0,0,2,2,2,2,2,2,2,2,55,10,2750,6,2,1,1,1,1,31,88,88,8,0,0,0,88,0,0,88,0,1,1,&,0,0,88,88,888,0,0,0,88,88,888,888,0,0,99,88,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,2,0
3,1972,1,0,1,1,1,01002,999,9,128,1,6,63,1,1002,6,63,2,3,9,2,2,3,16,3,4,4,2,2,1,1,0,0,2,2,2,2,2,2,2,2,99,11,2948,6,2,1,1,1,1,15,88,88,8,0,0,0,88,0,0,88,0,2,2,&,0,0,88,88,888,0,0,0,88,88,888,888,0,0,99,88,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,2,0
4,1972,1,0,1,1,1,01002,999,9,128,1,6,63,1,1002,6,63,1,1,1,1,1,1,19,6,7,7,2,2,1,1,0,0,2,2,2,2,2,2,2,2,22,3,3119,7,2,1,1,1,1,10,88,88,8,0,0,0,88,0,0,88,0,1,1,&,0,0,88,88,888,0,0,0,88,88,888,888,0,0,99,88,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,2,0


In [24]:
natality_1973_df = natality_dfs_68_78[5]
natality_1973_df.head()

,datayear,shipnum,reparea,rectype,restatus,stateres,cntyres,cityres,citrspop,smsares,metrores,divres,stsubres,statenat,cntynat,divocc,stsubocc,csex,birattnd,frace,mrace,crace,crace3,dmage,mage36,mage15,mage12,mage8,mage7,mage6,nlbnl,nlbnd,nlbd,dtotord,totord9,dlivord,livord9,livord8,livord7,livord6,livord3,dfage,fage11,dbirwt,birwt12,birwt3,dplural,plural3,plural2,birmon,birday,lmpmon,lmpday,lmpyr,dgestat,gestat10,gestat3,dmeduc,meduc14,meduc6,dfeduc,feduc14,dlegit,legit3,monprec,mpre10,mpre6,llbmon,llbyr,disllb,isllb17,isllb10,isllb8,lfdmon,lfdyr,dlfd,dterm,term9,outcome,mplbir,nprevis,legitstrflg,educstrflg,lmprflg,monprerflg,llbrflg,lfdrflg,legitsmsarflg,educsmsarflg,congenrflg,nprevisrflg,injuryrflg,legitstoflg,educstoflg,lmpoflg,monpreoflg,llboflg,lfdoflg,congenflg,nprevisoflg,injuryoflg,recwt,nprev28
0,1973,ç1,\,1,1,1,1037,999,9,26,1,6,63,1,1037,6,63,1,1,1,1,1,1,22,9,11,8,3,3,2,0,0,0,1,1,1,1,1,1,1,1,23,3,2977,6,2,1,1,1,10,27,88,88,8,0,0,0,88,0,0,88,0,1,1,&,0,0,88,88,888,0,0,0,88,88,888,888,0,0,1,88,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,2,0
1,1973,01,0,1,1,1,1001,999,9,0,2,6,63,1,1001,6,63,1,3,2,2,2,3,43,30,17,12,7,7,6,7,2,99,99,9,10,8,7,6,5,2,48,8,3629,8,2,1,1,1,1,1,88,88,8,0,0,0,88,0,0,88,0,1,1,&,0,0,88,88,888,0,0,0,88,88,888,888,0,0,1,88,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,2,0
2,1973,01,0,1,1,1,1002,999,9,128,1,6,63,1,1002,6,63,2,1,2,2,2,3,19,6,7,7,2,2,1,0,0,0,1,1,1,1,1,1,1,1,19,2,2552,6,2,1,1,1,1,22,88,88,8,0,0,0,88,0,0,88,0,1,1,&,0,0,88,88,888,0,0,0,88,88,888,888,0,0,1,88,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,2,0
3,1973,01,0,1,1,1,1002,999,9,128,1,6,63,1,1002,6,63,2,1,1,1,1,1,25,12,14,9,4,4,3,1,0,0,2,2,2,2,2,2,2,2,27,4,3062,7,2,1,1,1,1,27,88,88,8,0,0,0,88,0,0,88,0,1,1,&,0,0,88,88,888,0,0,0,88,88,888,888,0,0,11,88,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,2,0
4,1973,01,0,1,1,1,1002,999,9,128,1,6,63,1,1002,6,63,1,1,9,1,1,1,17,4,5,5,2,2,1,0,0,0,1,1,1,1,1,1,1,1,99,11,3147,7,2,1,1,1,1,29,88,88,8,0,0,0,88,0,0,88,0,2,2,&,0,0,88,88,888,0,0,0,88,88,888,888,0,0,1,88,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,2,0


In [31]:
# Make a copy of the list and keep only relevant columns in each DataFrame
# Convert cntyres and cityres to string to avoid type conversion issues
natality_small_dfs_68_78 = [df[['datayear', 'cntyres', 'cityres']].copy() for df in natality_dfs_68_78]
for df in natality_small_dfs_68_78:
    df['cntyres'] = df['cntyres'].astype(str)
    df['cityres'] = df['cityres'].astype(str)

# Concatenate all smaller DataFrames into a single DataFrame
nvss_68_79_small = pd.concat(natality_small_dfs_68_78, ignore_index=True)

# Ensure the output directory exists
output_dir = "datastore/output/nvss"
os.makedirs(output_dir, exist_ok=True)

# Save the concatenated dataframe to Parquet
nvss_68_79_small.to_parquet(os.path.join(output_dir, "nvss_68_79.parquet"), index=False)

In [4]:
output_dir = "datastore/output/nvss"
pd.read_parquet(os.path.join(output_dir, "nvss_68_79.parquet"))

,datayear,cntyres,cityres
0,1968,1020,999
1,1968,1001,999
2,1968,1001,999
3,1968,1026,999
4,1968,1001,999
...,...,...,...
23153992,1978,51004,999
23153993,1978,51014,999
23153994,1978,51018,999
23153995,1978,51007,999


In [5]:
data_dir = "datastore/raw/nvss/data"
natality_dfs_79_89= []

for year in range(1979, 1989):  # inclusive 1979 through 1989
    filename = f"natality{year}us.csv"
    filepath = os.path.join(data_dir, filename)
    if os.path.exists(filepath):
        df = pd.read_csv(filepath)
        df['datayear'] = year  # ensure datayear matches the file year
        natality_dfs_79_89.append(df)
    else:
        print(f"File {filename} does not exist.")


/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_77145/1448024086.py:8: DtypeWarning: Columns (1,6,7,8,9,10,54) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_77145/1448024086.py:8: DtypeWarning: Columns (1,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_77145/1448024086.py:8: DtypeWarning: Columns (1,6,7,8,9,10,117) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_77145/1448024086.py:8: DtypeWarning: Columns (6,7,8,9,10,65) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_77145/1448024086.py:8: DtypeWarning: Columns 

In [ ]:
# Make a copy of the list and keep only relevant columns in each DataFrame
# Convert cntyres and cityres to string to avoid type conversion issues
natality_small_dfs_79_89 = [df[['datayear', 'cntyres', 'cityres']].copy() for df in natality_dfs_79_89]
for df in natality_small_dfs_79_89:
    df['cntyres'] = df['cntyres'].astype(str)
    df['cityres'] = df['cityres'].astype(str)

# Concatenate all smaller DataFrames into a single DataFrame
nvss_79_89_small = pd.concat(natality_small_dfs_79_89, ignore_index=True)

# Ensure the output directory exists
output_dir = "datastore/output/nvss"
os.makedirs(output_dir, exist_ok=True)

# Save the concatenated dataframe to Parquet
nvss_79_89_small.to_parquet(os.path.join(output_dir, "nvss_79_88.parquet"), index=False)

In [10]:
pd.read_parquet(os.path.join(output_dir, "nvss_79_88.parquet"))

,datayear,cntyres,cityres
0,1979,01002,999
1,1979,01002,999
2,1979,01002,999
3,1979,01002,999
4,1979,01002,999
...,...,...,...
35142106,1988,51011,999
35142107,1988,51011,999
35142108,1988,51011,999
35142109,1988,51015,999


In [ ]:
# Make a copy of the list and keep only relevant columns in each DataFrame
# Convert cntyres and cityres to string to avoid type conversion issues
natality_small_dfs_68_78 = [df[['datayear', 'cntyres', 'cityres']].copy() for df in natality_dfs_68_78]
for df in natality_small_dfs_68_78:
    df['cntyres'] = df['cntyres'].astype(str)
    df['cityres'] = df['cityres'].astype(str)

# Concatenate all smaller DataFrames into a single DataFrame
nvss_68_79_small = pd.concat(natality_small_dfs_68_78, ignore_index=True)

# Ensure the output directory exists
output_dir = "datastore/output/nvss"
os.makedirs(output_dir, exist_ok=True)

# Save the concatenated dataframe to Parquet
nvss_68_79_small.to_parquet(os.path.join(output_dir, "nvss_68_79.parquet"), index=False)

In [3]:
data_dir = "datastore/raw/nvss/data"
natality_dfs_89_93 = []

for year in range(1989, 1994):  # inclusive 1989 through 1999
    filename = f"natality{year}us.csv"
    filepath = os.path.join(data_dir, filename)
    if os.path.exists(filepath):
        df = pd.read_csv(filepath)
        df['datayear'] = year  # ensure datayear matches the file year
        natality_dfs_89_93.append(df)
    else:
        print(f"File {filename} does not exist.")

/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_85249/464400410.py:8: DtypeWarning: Columns (21,22,23,24,28,203) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_85249/464400410.py:8: DtypeWarning: Columns (21,22,23,24,29,204,205) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_85249/464400410.py:8: DtypeWarning: Columns (21,22,23,24,29,204,205) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_85249/464400410.py:8: DtypeWarning: Columns (21,22,23,24,29,206,207) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_85249/464400410.py:8:

In [4]:
len(natality_dfs_89_93)

5

In [5]:
# Make a copy of the list and keep only relevant columns in each DataFrame
# Convert cntyres and cityres to string to avoid type conversion issues
natality_small_dfs_89_93 = [df[['datayear', 'cntyres', 'cityres']].copy() for df in natality_dfs_89_93]
for df in natality_small_dfs_89_93:
    df['cntyres'] = df['cntyres'].astype(str)
    df['cityres'] = df['cityres'].astype(str)

# Concatenate all smaller DataFrames into a single DataFrame
nvss_89_93_small = pd.concat(natality_small_dfs_89_93, ignore_index=True)

# Ensure the output directory exists
output_dir = "datastore/output/nvss"
os.makedirs(output_dir, exist_ok=True)

output_dir = "datastore/output/nvss"
nvss_89_93_small.to_parquet(os.path.join(output_dir, "nvss_89_93.parquet"), index=False)

In [6]:
data_dir = "datastore/raw/nvss/data"
natality_dfs_94_95 = []

for year in range(1994, 1996):  # inclusive 1989 through 1999
    filename = f"natality{year}us.csv"
    filepath = os.path.join(data_dir, filename)
    if os.path.exists(filepath):
        df = pd.read_csv(filepath)
        df['datayear'] = year  # ensure datayear matches the file year
        natality_dfs_94_95.append(df)
    else:
        print(f"File {filename} does not exist.")

/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_85249/646699947.py:8: DtypeWarning: Columns (21,22,23,24,30,205,206) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/j4/z817ljzd6w99bjqkz9bgh5mh0000gn/T/ipykernel_85249/646699947.py:8: DtypeWarning: Columns (21,22,23,24,30,198,199) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)


In [7]:
# Make a copy of the list and keep only relevant columns in each DataFrame
# Convert cntyres and cityres to string to avoid type conversion issues
natality_small_dfs_94_95  = [df[['datayear', 'cntyres', 'cityres']].copy() for df in natality_dfs_94_95]
for df in natality_small_dfs_94_95:
    df['cntyres'] = df['cntyres'].astype(str)
    df['cityres'] = df['cityres'].astype(str)

# Concatenate all smaller DataFrames into a single DataFrame
nvss_94_95_small = pd.concat(natality_small_dfs_94_95, ignore_index=True)

# Ensure the output directory exists
output_dir = "datastore/output/nvss"
os.makedirs(output_dir, exist_ok=True)

output_dir = "datastore/output/nvss"
nvss_94_95_small.to_parquet(os.path.join(output_dir, "nvss_94_95.parquet"), index=False)